# Backtesting a Strategy

We are ready to backtest a simple trading strategy with **PyBroker**! Let's begin by importing the needed classes below:

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

[Yahoo Finance](https://finance.yahoo.com) will be used as the [DataSource]() of our backtest. We also enable data source caching so that data is only downloaded one time when running backtests. 

The next step is to instantiate a new [Strategy]() instance that will be used to backtest our trading strategy:

In [2]:
config = StrategyConfig(initial_cash=500_000)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)

The above creates a ```Strategy``` that will download data between ```3/1/2017``` and ```3/1/2022``` from Yahoo Finance before running a backtest. A [StrategyConfig]() is used to configure the ```Strategy```. By default, the [initial_cash]() to use for the backtest would be ```100_000```, but the config overrides the default with a value of ```500_000```.

## Defining Strategy Rules

Now, let's implement a basic trading strategy with the following rules:

- Buy shares in a stock if:

    - There is no open long position in that stock.
    
    - The last close price is less than the low of the previous bar.
    
- Set the limit price of the buy order to ```0.01``` less than the last close price.
- Hold the position for 3 days before liquidating it at the current market price.
- Trade the rules on ```AAPL``` and ```MSFT```, allocating up to 25% of the portfolio to each.

In [3]:
def buy_low(ctx):
    # If shares were already purchased and are currently being held, then return.
    if ctx.long_pos():
        return
    # If the latest close price is less than the previous day's low price,
    # then place a buy order.
    if len(ctx.low) >= 2 and ctx.close[-1] < ctx.low[-2]:
        # Buy a number of shares that is equal to 25% the portfolio.
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        # Set the limit price of the order.
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        # Hold the position for 3 bars before liquidating (in this case, 3 days).
        ctx.hold_bars = 3

The ```buy_low``` rules are then added to the ```Strategy``` for ```AAPL``` and ```MSFT``` using [add_execution]():

In [4]:
strategy.add_execution(buy_low, ['AAPL', 'MSFT'])

That is a lot to unpack! To fully understand how the above works, we need to step through **PyBroker's** execution model for running backtests.

When the backtest runs, the ```buy_low``` function will be called separately for ```AAPL``` and ```MSFT``` on every bar of data. Here, each bar corresponds to a single day of data for the ticker that was retrieved from Yahoo Finance.

An [ExecContext]() (```ctx```) will be passed to ```buy_low```. The ```ExecContext``` will contain data for the current ticker symbol being executed using the trading rules in ```buy_low```. For instance, [ctx.close]() will contain all of the close prices up until the most recent bar of the current ticker symbol (```AAPL``` or ```MSFT```). The latest close price is retrieved with ```ctx.close[-1]```.

The ```ExecContext``` is also used to place a buy order. The number of shares to purchase is set in ```buy_low``` using [ctx.buy_shares](), and the number of shares is calculated with [ctx.target_shares](). In this case, the number of shares to buy will be equal to 25% of the portfolio.

The limit price of the order is set by [buy_limit_price](). If satisifed, the buy order will be filled on the next bar. The time when the order is filled can be configured with [StrategyConfig.buy_delay](), and its fill price can be set with [ExecContext.buy_fill_price](). By default, buy orders are filled on the next bar (```buy_delay=1```) and at a [fill price equal to the midpoint between that bar's open and close price]().

Finally, [ctx.hold_bars]() specifies how many bars to hold the position before liquidating it. When liquidating, the shares are sold at market price equal to [ExecContext.sell_fill_price](), which is configurable and also defaults to the midpoint between the bar's open and close price.

The ```ExecContext``` contains many other fields containing information about current positions, orders, OHLC prices and more. [You can read more on the reference documentation of ExecContext]().

## Adding a Second Execution

We are not limited to only using one set of trading rules for one group of tickers. Another execution that implements different trading rules for different tickers can be added to the ```Strategy``` .

Let's implement another set of trading rules that are similar as before, but are for a short strategy:

In [5]:
def short_high(ctx):
    # If shares were already shorted and that position is currently being held, then return.
    if ctx.short_pos():
        return
    # If the latest close price is more than the previous day's high price,
    # then place a sell order.
    if len(ctx.high) >= 2 and ctx.close[-1] > ctx.high[-2]:
        # Short 100 shares.
        ctx.sell_shares = 100
        # Cover the shares after 2 bars (in this case, 2 days).
        ctx.hold_bars = 2

And let's trade these rules on ```TSLA```:

In [6]:
strategy.add_execution(short_high, ['TSLA'])

## Running a Backtest

Running a backtest is as simple as calling the [backtest]() method on the ```Strategy```:

In [7]:
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2019-08-29 00:00:00 to 2022-02-25 00:00:00


100% (629 of 629) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:03


That was fast! Notice that an argument [calc_bootstrap=False]() was passed in to disable calculating bootstrap metrics. We can ignore that for now until the next notebook about bootstrap metrics.

The ```result``` returned is an instance of [TestResult](). Let's take a look at the results, starting with the daily balances of our portfolio:

In [8]:
result.portfolio

,cash,equity,margin,market_value,pnl,unrealized_pnl
date,,,,,,
2019-08-29,500000.00,500000.00,0.00,500000.00,0.00,0.00
2019-08-30,500000.00,500000.00,0.00,500000.00,0.00,0.00
2019-09-03,501507.00,501507.00,1500.07,501513.93,0.00,6.93
2019-09-04,501507.00,501507.00,1471.20,501542.80,0.00,35.80
2019-09-05,500005.00,500005.00,0.00,500005.00,5.00,0.00
...,...,...,...,...,...,...
2022-02-18,293257.66,610674.79,0.00,610674.79,112865.38,-2190.59
2022-02-22,293257.66,607731.07,0.00,607731.07,112865.38,-5134.31
2022-02-23,293257.66,599592.86,0.00,599592.86,112865.38,-13272.52


Likewise, the result contains the daily balance of each position that was held:

In [9]:
result.positions

long_shares  short_shares   close     equity  market_value  \
symbol date                                                                     
TSLA   2019-09-03            0           100   15.00       0.00          6.93   
       2019-09-04            0           100   14.71       0.00         35.80   
       2019-09-06            0           100   15.16       0.00         -0.33   
       2019-09-09            0           100   15.45       0.00        -29.27   
MSFT   2019-09-10          911             0  136.08  123968.88     123968.88   
...                        ...           ...     ...        ...           ...   
AAPL   2022-02-18          949             0  167.30  158767.70     158767.70   
MSFT   2022-02-22          551             0  287.72  158533.72     158533.72   
AAPL   2022-02-22          949             0  164.32  155939.69     155939.69   
MSFT   2022-02-23          551             0  280.27  154428.76     154428.76   
AAPL   2022-02-23          949             0  160.07  151906.44     151906.44   

                    margin  unrealized_pnl  
symbol date                                 
TSLA   2019-09-03  1500.07            6.93  
       2019-09-04  1471.20           35.80  
       2019-09-06  1516.33           -0.33  
       2019-09-09  1545.27          -29.27  
MSFT   2019-09-10     0.00          346.18  
...                    ...             ...  
AAPL   2022-02-18     0.00        -1005.94  
MSFT   2022-02-22     0.00        -1300.36  
AAPL   2022-02-22     0.00        -3833.95  
MSFT   2022-02-23     0.00        -5405.32  
AAPL   2022-02-23     0.00        -7867.20  

[485 rows x 7 columns]

And all of the orders that were placed, too:

In [10]:
result.orders

,date,symbol,order_type,limit_price,fill_price,shares,pnl,pnl %
id,,,,,,,,
1,2019-09-03,TSLA,sell,NaN,15.07,100,0.00,0.000000
2,2019-09-05,TSLA,buy,NaN,15.02,100,5.00,24.975025
3,2019-09-06,TSLA,sell,NaN,15.16,100,0.00,0.000000
4,2019-09-10,TSLA,buy,NaN,15.48,100,-32.00,-67.396799
5,2019-09-10,MSFT,buy,137.51,135.70,911,0.00,0.000000
...,...,...,...,...,...,...,...,...
398,2022-02-18,TSLA,buy,NaN,287.41,100,1720.00,85.682546
399,2022-02-18,MSFT,buy,290.72,290.08,551,0.00,0.000000
400,2022-02-18,AAPL,buy,168.87,168.36,949,0.00,0.000000


The result also contains a DataFrame of metrics calculated on the out-of-sample results of the backtest:

In [11]:
result.metrics_df

,name,value
0,trade_count,402.000000
1,initial_value,500000.000000
2,end_value,598779.070000
3,total_profit,245219.260000
4,total_loss,-146440.190000
5,max_drawdown,-54071.400000
6,max_drawdown_pct,-8.346543
7,win_rate,0.547264
8,loss_rate,0.452736
9,avg_profit,2229.266000


[You can read about what these metrics mean here]().

## Filtering Backtest Data

Lastly, it is possible to filter the data used for the backtest. For example, the data can be filtered to only contain bars for Mondays. This, in effect, limits the strategy to only trade on Mondays:

In [12]:
result = strategy.backtest(days='mon', train_size=0.5, calc_bootstrap=False)
result.metrics_df

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2019-09-09 00:00:00 to 2022-02-28 00:00:00


100% (119 of 119) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,name,value
0,trade_count,96.000000
1,initial_value,500000.000000
2,end_value,524766.210000
3,total_profit,92139.090000
4,total_loss,-70520.750000
5,max_drawdown,-51841.520000
6,max_drawdown_pct,-10.186922
7,win_rate,0.510638
8,loss_rate,0.489362
9,avg_profit,3839.128750


Note that the data did not need to be downloaded again from Yahoo Finance, since ```DataSource``` caching was enabled and because the cached data only needed to be filtered.

From the above, it seems like we already have a profitable strategy. But not so fast! We may be getting fooled by randomness. [Next, we will learn about using bootstrapping to aide in evaluating our trading strategies]().